In [20]:
import pandas as pd
import sys,os,copy,pdb,importlib
sys.path.append('../classes')
sys.path.append('../analysis')
import numpy as np
import matplotlib.pyplot as plt

import tournament_new as tn
import create_database as cd
import repeated_tournament as rt
importlib.reload(tn)

from beck.beck_game import BeckGame as Game

# new_tournament_dir = tn.NEW_TOURNAMENT_DIR
new_tournament_dir = rt.NEW_TOURNAMENT_DIR
# old_tournament_loc =tn.TOURNAMENT_BASE_LOC
old_tournament_loc =rt.TOURNAMENT_BASE_LOC

In [2]:
old_res = pd.read_pickle(old_tournament_loc)

In [4]:
for fn in os.listdir(new_tournament_dir):
    f_loc=os.path.join(new_tournament_dir,fn)
    res = pd.read_pickle(f_loc)
    old_res.update(res)

In [8]:
mask=['tournament_6' in n for n in old_res.index]
old_res.loc[mask]

id                                                 tournament_1;mcts25;cpuct1;1  \
                                                                        row_win   
id                                                                                
tournament_6;mcts100;cpuct2;id-2;1                                          5.0   
tournament_6;mcts100;cpuct2;id-1;1                                          5.0   
tournament_6;mcts100;cpuct2;id-0;1                                          5.0   
tournament_6;mcts100;cpuct2;id-0;2                                          5.0   
tournament_6;mcts100;cpuct2;id-1;2                                          5.0   
...                                                                         ...   
tournament_6;mcts100;cpuct2;id-0;50;bfts100;prune2                          0.0   
cog_id_1;tournament_6;mcts100;cpuct2;id-0;51                                5.0   
tournament_6;mcts100;cpuct2;id-0;51;bfts100;prune2                          0.0   
cog_id_1;tournament_6;mcts100;cpuct2;id-0;52                                5.0   
tournament_6;mcts100;cpuct2;id-0;52;bfts100;prune2                          0.0   

id                                                               \
                                                   col_win draw   
id                                                                
tournament_6;mcts100;cpuct2;id-2;1                     0.0  0.0   
tournament_6;mcts100;cpuct2;id-1;1                     0.0  0.0   
tournament_6;mcts100;cpuct2;id-0;1                     0.0  0.0   
tournament_6;mcts100;cpuct2;id-0;2                     0.0  0.0   
tournament_6;mcts100;cpuct2;id-1;2                     0.0  0.0   
...                                                    ...  ...   
tournament_6;mcts100;cpuct2;id-0;50;bfts100;prune2     0.0  0.0   
cog_id_1;tournament_6;mcts100;cpuct2;id-0;51           0.0  0.0   
tournament_6;mcts100;cpuct2;id-0;51;bfts100;prune2     0.0  0.0   
cog_id_1;tournament_6;mcts100;cpuct2;id-0;52           0.0  0.0   
tournament_6;mcts100;cpuct2;id-0;52;bfts100;prune2     0.0  0.0   

id                                                 tournament_1;mcts25;cpuct1;2  \
                                                                        row_win   
id                                                                                
tournament_6;mcts100;cpuct2;id-2;1                                          0.0   
tournament_6;mcts100;cpuct2;id-1;1                                          0.0   
tournament_6;mcts100;cpuct2;id-0;1                                          5.0   
tournament_6;mcts100;cpuct2;id-0;2                                          0.0   
tournament_6;mcts100;cpuct2;id-1;2                                          5.0   
...                                                                         ...   
tournament_6;mcts100;cpuct2;id-0;50;bfts100;prune2                          0.0   
cog_id_1;tournament_6;mcts100;cpuct2;id-0;51                                5.0   
tournament_6;mcts100;cpuct2;id-0;51;bfts100;prune2                          0.0   
cog_id_1;tournament_6;mcts100;cpuct2;id-0;52                                5.0   
tournament_6;mcts100;cpuct2;id-0;52;bfts100;prune2                          0.0   

id                                                               \
                                                   col_win draw   
id                                                                
tournament_6;mcts100;cpuct2;id-2;1                     5.0  0.0   
tournament_6;mcts100;cpuct2;id-1;1                     5.0  0.0   
tournament_6;mcts100;cpuct2;id-0;1                     0.0  0.0   
tournament_6;mcts100;cpuct2;id-0;2                     5.0  0.0   
tournament_6;mcts100;cpuct2;id-1;2                     0.0  0.0   
...                                                    ...  ...   
tournament_6;mcts100;cpuct2;id-0;50;bfts100;prune2     0.0  0.0   
cog_id_1;tournament_6;mcts100;cpuct2;id-0;51           0.0  0.0  

In [12]:
((old_res!=5)&(old_res!=0)).sum().sum()

0

In [13]:
base_result_df = copy.copy(old_res)

In [17]:
all_p = pd.read_pickle(cd.DATABASE_LOC)

In [64]:
importlib.reload(rt)

<module 'repeated_tournament' from '../classes/repeated_tournament.py'>

In [68]:
p1id = ';bfts100;prune2;cog_id_1'#'cog_id_1;tournament_6;mcts100;cpuct2;id-0;51'#
p2id = 'tournament_6;mcts100;cpuct2;id-0;51'

p1 = all_p.loc[all_p.id==p1id].iloc[0]
p2 = all_p.loc[all_p.id==p2id].iloc[0]
g = Game(4,9,4)


rep=5
for r in range(rep):
#     if base_result_df.loc[p1.id, p2.id].sum()>=rep: # enough reps recorded
#         break
    print(f'rep {r}')
    one_res = rt.play_game(g, p1, p2)
    print(f'res {one_res}')
    if one_res == 1:
        base_result_df.loc[p1.id, (p2.id,'row_win')] += 1 
    elif one_res == -1:
        base_result_df.loc[p1.id, (p2.id,'col_win')] += 1 
    elif one_res == 0.0001:
        base_result_df.loc[p1.id, (p2.id,'draw')] += 1 
    else:
        printl('game result not understood')

rep 0


Game beginning! ;bfts100;prune2;cog_id_1 v/s tournament_6;mcts100;cpuct2;id-0;51...
res 1
rep 1


Game beginning! ;bfts100;prune2;cog_id_1 v/s tournament_6;mcts100;cpuct2;id-0;51...
res 1
rep 2


Game beginning! ;bfts100;prune2;cog_id_1 v/s tournament_6;mcts100;cpuct2;id-0;51...
res 1
rep 3


Game beginning! ;bfts100;prune2;cog_id_1 v/s tournament_6;mcts100;cpuct2;id-0;51...
res 1
rep 4


Game beginning! ;bfts100;prune2;cog_id_1 v/s tournament_6;mcts100;cpuct2;id-0;51...
res 1


In [55]:
base_result_df.loc[p1.id, p2.id]

row_win    0.0
col_win    0.0
draw       5.0
Name: cog_id_1;tournament_6;mcts100;cpuct2;id-0;51, dtype: float64

In [53]:
all_p.loc[all_p.tournament.isna()]

,tree_type,value_func_type,other_type,n_mcts,cpuct,value_func_iter,mcts_iter,n_bfts,pruning_thresh,center,2_con,2_uncon,3,4,oppo_scale,id,tournament,mcts_location,value_func_location,better_than_last
964,bfts,cog,NaN,NaN,NaN,NaN,NaN,100,2,0.01,0.2,0.05,2,100,0.1,;bfts100;prune2;cog_id_1,NaN,NaN,NaN,NaN
965,bfts,cog,NaN,NaN,NaN,NaN,NaN,100,1,0.01,0.2,0.05,2,100,0.1,;bfts100;prune1;cog_id_1,NaN,NaN,NaN,NaN


In [62]:
p1.id

'cog_id_1;tournament_6;mcts100;cpuct2;id-0;51'

In [60]:
aa=base_result_df.loc[p1.id].sum(level=0)
aa.loc[aa==0]
mask=['tournament_4' not in id for id in aa.index]
aa.loc[(aa==0)&(mask)]

id
;bfts100;prune2;cog_id_1                              0.0
;bfts100;prune1;cog_id_1                              0.0
cog_id_1;tournament_1;mcts100;cpuct2;21               0.0
tournament_1;mcts100;cpuct2;21;bfts100;prune2         0.0
tournament_1;mcts100;cpuct2;21;bfts100;prune1         0.0
                                                     ... 
tournament_7;mcts100;cpuct2;id-0;33;bfts100;prune2    0.0
cog_id_1;tournament_7;mcts100;cpuct2;id-0;38          0.0
tournament_7;mcts100;cpuct2;id-0;38;bfts100;prune2    0.0
cog_id_1;tournament_7;mcts100;cpuct2;id-0;52          0.0
tournament_7;mcts100;cpuct2;id-0;52;bfts100;prune2    0.0
Name: cog_id_1;tournament_6;mcts100;cpuct2;id-0;51, Length: 505, dtype: float64

In [26]:
old_res.to_pickle(os.path.join(new_tournament_dir,'repeated_round_robin_mistakenly_det.pkl'))

In [74]:
from arena import Arena
# p1_id = 'tournament_1;mcts100;cpuct1;39'
#'tournament_1;mcts50;cpuct1;best'#'tournament_1;mcts100;cpuct2;best'#'tournament_4;mcts100;cpuct2;id-3754964;best'#'cog_id_1;tournament_4;mcts100;cpuct2;id-3754964;best'#'cog_id_1;tournament_1;mcts100;cpuct2;21'
# p2_id = 'tournament_4;mcts100;cpuct2;id-3754964;best'#'tournament_1;mcts100;cpuct2;21'
# p2_id = 'tournament_1;mcts100;cpuct1;4'
#'tournament_1;mcts50;cpuct1;39'#'tournament_5;mcts100;cpuct2;id-0;best'#'tournament_1;mcts100;cpuct2;best'#'tournament_1;mcts100;cpuct2;21'

game = Game(4,9,4)
select_row_by_id = lambda id,df:df.loc[df['id']==id].iloc[0]
# p1_row = select_row_by_id(p1_id,all_p)
# p1_row = cd.merge_cog_with_mcts(all_p,'cog_id_1',p2_id)
# p2_row = select_row_by_id(p2_id,all_p)
p1_row=p1
p2_row=p2
p1,_,p1_tree = tn.get_player(game, p1_row, temp=1/10,deterministic=False)
p2,_,p2_tree = tn.get_player(game, p2_row, temp=1/10,deterministic=False)
arena = Arena(p1, p2, game, display=game.display,tree1=p1_tree,tree2=p2_tree)
res=arena.playGames(10,verbose=True)

Arena.playGames (1):   0%|          | 0/5 [00:00<?, ?it/s]

Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - O - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - O X - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  4 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - O O X - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  5 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - O O X - - - |
2 |- - X - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  6 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - O

Arena.playGames (1):  20%|██        | 1/5 [00:33<02:15, 33.87s/it]

Turn  19 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X O - - O O - - |
1 |- O X O O X - - - |
2 |- - X O X O X - - |
3 |- - X X O X - - - |
-----------------------
Turn  20 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X O - O O O - - |
1 |- O X O O X - - - |
2 |- - X O X O X - - |
3 |- - X X O X - - - |
-----------------------
Turn  21 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X O X O O O - - |
1 |- O X O O X - - - |
2 |- - X O X O X - - |
3 |- - X X O X - - - |
-----------------------
Game over: Turn  21 Result  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X O X O O O O - |
1 |- O X O O X - - - |
2 |- - X O X O X - - |
3 |- - X X O X - - - |
-----------------------
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - 

Arena.playGames (1):  40%|████      | 2/5 [01:13<01:46, 35.61s/it]

...

36 Result  0.0001
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X X O X O X X X O |
1 |X X O O X X X O O |
2 |X O O O X O O O X |
3 |O X X O O X X O O |
-----------------------
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - O - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - O X - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  4 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - O - - - - - |
1 |- - - - - - - - - |
2 |- - - O X - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  5 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - O - - - - - |
1 |- - - - 

Arena.playGames (2):  80%|████████  | 4/5 [02:59<00:44, 44.84s/it]

Turn  34 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |O X X O O X - O - |
1 |O O X O X O O X X |
2 |X O O O X O O X X |
3 |O X O X X O X X - |
-----------------------
Turn  35 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |O X X O O X X O - |
1 |O O X O X O O X X |
2 |X O O O X O O X X |
3 |O X O X X O X X - |
-----------------------
Turn  36 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |O X X O O X X O O |
1 |O O X O X O O X X |
2 |X O O O X O O X X |
3 |O X O X X O X X - |
-----------------------
Game over: Turn  36 Result  0.0001
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |O X X O O X X O O |
1 |O O X O X O O X X |
2 |X O O O X O O X X |
3 |O X O X X O X X X |
-----------------------
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - 

Arena.playGames (2): 100%|██████████| 5/5 [03:49<00:00, 45.90s/it]

Turn  36 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |O X X X O X O X X |
1 |O O O X X O X X O |
2 |X X O O O X O O X |
3 |- O X X O O O X O |
-----------------------
Game over: Turn  36 Result  0.0001
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |O X X X O X O X X |
1 |O O O X X O X X O |
2 |X X O O O X O O X |
3 |X O X X O O O X O |
-----------------------


In [75]:
res

(5, 0, 5)